In [2]:
import yaml
import typing
from googletrans import Translator
translator = Translator(service_urls=[
    'translate.google.cn',])

class Loader(yaml.SafeLoader):
    pass

class Tagged(typing.NamedTuple):
    tag: str
    value: object
    
def construct_undefined(loader, node):
    if isinstance(node, yaml.nodes.ScalarNode):
        value = loader.construct_scalar(node)
    elif isinstance(node, yaml.nodes.SequenceNode):
        value = loader.construct_sequence(node)
    elif isinstance(node, yaml.nodes.MappingNode):
        value = loader.construct_mapping(node)
    else:
        assert False, f"unexpected node: {node!r}"
    return Tagged(node.tag, value)

Loader.add_constructor(None, construct_undefined)

langs = [
    'ar', 
    'bg', 
    'da',
    'de', 
    'el', 
    'en', 
    'es', 
    'et', 
    'fa', 
    'fr', 
    'hi', 
    'hr', 
    'ht', 
    'id', 
    'it', 
    'ja', 
    'ko', 
    'nl', 
    'qu', 
    'ru', 
    'sw', 
    'ta', 
    'th', 
    'tr', 
    'ur', 
    'vi', 
    'zh-cn',
]

In [4]:
# gigaword
origin_file = "promptsource/templates/gigaword/en/templates.yaml"
origins = [
    ["Generate a title for this article:"],
    ["Title:"],
    ["Make a title for this article:"],
    ["First sentence of the article:", "Title:"],
    ["Too Long; Didn't Read"],
    ["Given the above sentence, write its title:"],
    ["Write a title for this sentence:", "Title:"],
    ["In a nutshell,"],
    ["Write an article with the given title:", "Title:"],
]
for i in range(17, len(langs)):
    lang = langs[i]
    if lang == 'en':
        continue
    tmpl = yaml.load(open(origin_file), Loader=Loader)
    tmpl["subset"] = lang[:2]
    i = 0
    for template in tmpl["templates"].values():
        origin = origins[i]
        for item in origin:
            trans=translator.translate(item, src='en', dest=lang)
            if item == "Too Long; Didn't Read":
                template.value['jinja'].replace('TL;DR', trans.text)
            else:
                template.value['jinja'].replace(trans.origin, trans.text)
        i += 1
    file = f"promptsource/templates/gigaword/{lang[:2]}/templates.yaml"
    yaml.dump(tmpl, file)

ConnectError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。